<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/src/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>Working with Upper Air Observations</h1>
    <h3>AMS 2023 Short Course: MetPy for your Data, Analyzing Meteorological Observations in Python</h3>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

### Tasks
1. <a href="#download">Access upper air data from the Iowa Environmental Mesonet Archive</a>
1. <a href="#plot">Activity: Visualize the data</a>
1. <a href="#download_profile">Download a profile of upper air data from the University of Wyoming Archive</a>
1. <a href="#mixingratio">Ploting with Skew-T Diagrams</a>

In this lesson, we will demonstrate how to use tools in MetPy and Siphon to easily access upper air observations and plot these data on a map or as a profile on a Skew-T.

## Access upper air data from the Iowa Environmental Mesonet Archive

In [ ]:
from datetime import datetime
from siphon.simplewebservice.iastate import IAStateUpperAir

orig_data = IAStateUpperAir.request_all_data(datetime(1993, 3, 13, 12), pressure=500)

In [ ]:
orig_data

Now that we have the data downloaded, we need to massage it a bit so we can work with it. Notice anything missing?

In [ ]:
from metpy.io import add_station_lat_lon

upper_air_df = add_station_lat_lon(orig_data)

In [ ]:
upper_air_df

So we now have added latitude/longitude information using MetPy's station tables. But we have two other problems. One is the stations for which our latitude and longitude information is missing. The other relates to the actual data type of the latitude and longitude.

In [ ]:
upper_air_df.info()

So to address this, we can use some Pandas tools:

In [ ]:
upper_air_df = upper_air_df.astype({'latitude': float, 'longitude': float})
upper_air_df = upper_air_df.dropna(subset=['latitude', 'longitude'])

For many use cases, this is enough massaging data. If we want to make use of the unit information, we can also use a tool from Metpy `pandas_dataframe_to_unit_arrays`, which takes unit information and converts the Pandas columns into MetPy's prototypical `pint.Quantity` (numpy arrays with unit information) that works with all of MetPy's calculations.

In [ ]:
from metpy.units import pandas_dataframe_to_unit_arrays

upper_air_data = pandas_dataframe_to_unit_arrays(upper_air_df, orig_data.units)

In [ ]:
upper_air_data

## Activity: Visualize the data

<div class="admonition alert alert-warning">

<p class="admonition-title" style="font-weight:bold">Activity: Visualize your data</p>

Now that we have the data, we can plot it just like we have other data sets today:    
    
* Set up a Cartopy-based map plot

* Add contours of the height using the `tricontour` plotting function (looks just like `contour` but can work with irregularly-spaced 2D data)
    
* Using what was done with the surface data, add some station plots of the data including:
    - Station ID
    - Height
    - Temperature
    - Dewpoint
    - Wind barbs

* Bonus: Add red contours of temperature as well

</div>

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from metpy.plots import StationPlot


# YOUR CODE BEGINS HERE



## Download a profile of upper air data from the University of Wyoming Archive

As an additional source for data, Siphon also has a class that simplifies access to the University of Wyoming's archive of upper air data. This uses an almost identical interface as the one used above.

In [ ]:
from siphon.simplewebservice.wyoming import WyomingUpperAir

df = WyomingUpperAir.request_data(datetime(2013, 5, 21, 0), 'DNR')

# Convert to arrays with units attached
sounding_data = pandas_dataframe_to_unit_arrays(df)

In [ ]:
sounding_data

## Plotting with Skew-T Diagrams
In meteorology, in addition to the 2D planar map plots, we often analyze upper air data by looking at vertical profiles, plotted on a Skew-T logP diagram. MetPy features tools to help generate these plots in Python.

From here, we can quickly generate a skew-T plot using MetPy's `SkewT` class:

In [ ]:
from metpy.plots import SkewT

skew = SkewT()
skew.plot(sounding_data['pressure'], sounding_data['temperature'], 'tab:red')

`SkewT` also features tools to add some common plot features, like dry adiabats or wind barbs:

In [ ]:
skew = SkewT(plt.figure(figsize=(12, 12)))
skew.plot(sounding_data['pressure'], sounding_data['temperature'], 'tab:red')
skew.plot_barbs(sounding_data['pressure'], sounding_data['u_wind'], sounding_data['v_wind'])

# Make the lines mostly transparent so that they're more subtle on the plot
skew.plot_dry_adiabats(alpha=0.2)

<div class="admonition alert alert-warning">

<p class="admonition-title" style="font-weight:bold">Activity: Visualize using a Skew-T log-p</p>

Let's practice making your own Skew-T:
    
* Download data for a date and site of your choosing. If you're not familiar with any sites, 'DNR' is the site ID for Denver, though you'll need to use older data since Denver has not launched a sounding recently.

* Set up a blank Skew-T

* Plot the profiles of temperature and dewpoint, as well as the wind barbs

* Customize the plot with special lines like dry and moist adiabats and lines of mixing ratio. You can [look at the docs](https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.SkewT.html#metpy.plots.SkewT) for more information

* BONUS: The `.ax` property on an instane of `SkewT` gives access to the underlying Matplotlib `Axes` object. Try using `skew.ax.axvline()` to plot lines that highlight the 0 and -20 degree isotherms.
    
</div>